In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

In [3]:
data = pd.read_csv('task_1/train.csv', sep=',')

In [4]:
# obj_id - id объекта, которому ставили оценку
# hour - часы для порядка оценок в рамках одного объекта. Между объектам эти значения нельзя сравнивать
# day - дни для порядка оценок в рамках одного объекта. Между объектам эти значения нельзя сравнивать
# vote - сама оценка
# old_user - категория насколько давно пользователь с нами,
# чем больше значение, тем больше истоии про пользователя у нас есть
data.head()

,obj_id,hour,day,vote,ip,old_user
0,1719,1905,79,5,1,5
1,3309,1904,79,9,3,5
2,2038,1903,79,8,4,5
3,225,1907,79,8,5,0
4,2009,1905,79,7,6,5


Разделим данные на 3 группы

In [11]:
new_data.head()

,obj_id,hour,day,vote,ip,old_user
0,1719,1905,79,2,1,2
1,3309,1904,79,3,1,2
2,2038,1903,79,3,2,2
3,225,1907,79,3,2,0
4,2009,1905,79,2,2,2


In [24]:
new_data = data.copy()
# new_data[new_data.vote==2] = 1
# new_data[new_data.vote==3] = 1
# new_data[new_data.vote==4] = 2
# new_data[new_data.vote==5] = 2
# new_data[new_data.vote==6] = 2
# new_data[new_data.vote==7] = 2
# new_data[new_data.vote==8] = 3
# new_data[new_data.vote==9] = 3
# new_data[new_data.vote==10] = 3
new_data[new_data==2] = 1
new_data[new_data==3] = 1
new_data[new_data==4] = 2
new_data[new_data==5] = 2
new_data[new_data==6] = 2
new_data[new_data==7] = 2
new_data[new_data==8] = 3
new_data[new_data==9] = 3
new_data[new_data==10] = 3
counts_data = new_data.pivot_table(index=['day', 'obj_id'], values='vote', aggfunc='count').reset_index()
min_border_day = np.percentile(counts_data.vote, 70)
min_border_obj = np.percentile(np.array(counts_data.vote), 99.5)

In [19]:
new_data[new_data.vote==2] = 1
new_data[new_data.vote==3] = 1
new_data[new_data.vote==4] = 2
new_data[new_data.vote==5] = 2
new_data[new_data.vote==6] = 2
new_data[new_data.vote==7] = 2
new_data[new_data.vote==8] = 3
new_data[new_data.vote==9] = 3
new_data[new_data.vote==10] = 3

counts_data = new_data.pivot_table(index=['day', 'obj_id'], values='vote', aggfunc='count').reset_index()
min_border_day = np.percentile(counts_data.vote, 70)
min_border_obj = np.percentile(np.array(counts_data.vote), 99.5)

In [20]:
min_border_obj

27.0

In [70]:
obj_id = 16
data_obj = new_data[new_data.obj_id == obj_id]
counts_days = data_obj.pivot_table(index=['day', 'obj_id'], values='vote', aggfunc='count').reset_index()
if np.max(counts_days.vote) > min_border_obj:
    data_obj_counts = data_obj.merge(counts_days, on=['day', 'obj_id'], how='inner')
    data_obj_counts = data_obj_counts[data_obj_counts.vote_y > min_border_day]
    counts_days = counts_days[counts_days.vote > min_border_day]
    
    group_1 = data_obj_counts[data_obj_counts.vote_x == 1].pivot_table(index=['day', 'obj_id'], values='vote_x', aggfunc='count').reset_index()
    group_2 = data_obj_counts[data_obj_counts.vote_x == 2].pivot_table(index=['day', 'obj_id'], values='vote_x', aggfunc='count').reset_index()
    group_3 = data_obj_counts[data_obj_counts.vote_x == 3].pivot_table(index=['day', 'obj_id'], values='vote_x', aggfunc='count').reset_index()
    data_obj_counts = data_obj_counts.merge(group_1, on=['day', 'obj_id'], how='inner')
    data_obj_counts = data_obj_counts.merge(group_2, on=['day', 'obj_id'], how='inner')
    data_obj_counts = data_obj_counts.merge(group_3, on=['day', 'obj_id'], how='inner')

    counts_days = counts_days.merge(group_1, on=['day', 'obj_id'], how='inner')
    counts_days = counts_days.merge(group_2, on=['day', 'obj_id'], how='inner')
    counts_days = counts_days.merge(group_3, on=['day', 'obj_id'], how='inner')

    

#         counts_days['rate_1'] = counts_days.vote_x_x / counts_days.vote
#         counts_days['rate_2'] = counts_days.vote_x_y / counts_days.vote
#         counts_days['rate_3'] = counts_days.vote_x / counts_days.vote

#         features = [np.max(counts_days.rate_1), np.mean(counts_days.rate_1), np.min(counts_days.rate_1)]
#         features += [np.max(counts_days.rate_2), np.mean(counts_days.rate_2), np.min(counts_days.rate_2)]
#         features += [np.max(counts_days.rate_3), np.mean(counts_days.rate_3), np.min(counts_days.rate_3)]

#         all_features.append(features)

#         if obj_id in true_answers:
#             all_answers.append(1)
#         else:
#             all_answers.append(0)

In [73]:
group_2

,day,obj_id


In [71]:
counts_days.head()

,vote,vote_x,day,obj_id


In [66]:
true_answers = [792, 1710, 423, 598, 364, 453, 16]
all_features = []
all_answers = []


for obj_id in tqdm(np.sort(new_data.obj_id.unique())):
    data_obj = new_data[new_data.obj_id == obj_id]
    counts_days = data_obj.pivot_table(index=['day', 'obj_id'], values='vote', aggfunc='count').reset_index()
    if np.max(counts_days.vote) > min_border_obj:
        data_obj_counts = data_obj.merge(counts_days, on=['day', 'obj_id'], how='inner')
        data_obj_counts = data_obj_counts[data_obj_counts.vote_y > min_border_day]
        counts_days = counts_days[counts_days.vote > min_border_day]

        group_1 = data_obj_counts[data_obj_counts.vote_x == 1].pivot_table(index=['day', 'obj_id'], values='vote_x', aggfunc='count').reset_index()
        group_2 = data_obj_counts[data_obj_counts.vote_x == 2].pivot_table(index=['day', 'obj_id'], values='vote_x', aggfunc='count').reset_index()
        group_3 = data_obj_counts[data_obj_counts.vote_x == 3].pivot_table(index=['day', 'obj_id'], values='vote_x', aggfunc='count').reset_index()
        data_obj_counts = data_obj_counts.merge(group_1, on=['day', 'obj_id'], how='inner')
        data_obj_counts = data_obj_counts.merge(group_2, on=['day', 'obj_id'], how='inner')
        data_obj_counts = data_obj_counts.merge(group_3, on=['day', 'obj_id'], how='inner')
        
        counts_days = counts_days.merge(group_1, on=['day', 'obj_id'], how='inner')
#         print(counts_days.columns)
        counts_days = counts_days.merge(group_2, on=['day', 'obj_id'], how='inner')
        counts_days = counts_days.merge(group_3, on=['day', 'obj_id'], how='inner')
        break
counts_days.head()
        
#         counts_days['rate_1'] = counts_days.vote_x_x / counts_days.vote
#         counts_days['rate_2'] = counts_days.vote_x_y / counts_days.vote
#         counts_days['rate_3'] = counts_days.vote_x / counts_days.vote

#         features = [np.max(counts_days.rate_1), np.mean(counts_days.rate_1), np.min(counts_days.rate_1)]
#         features += [np.max(counts_days.rate_2), np.mean(counts_days.rate_2), np.min(counts_days.rate_2)]
#         features += [np.max(counts_days.rate_3), np.mean(counts_days.rate_3), np.min(counts_days.rate_3)]

#         all_features.append(features)
        
#         if obj_id in true_answers:
#             all_answers.append(1)
#         else:
#             all_answers.append(0)

  0%|          | 0/2252 [00:00<?, ?it/s]


,vote,vote_x,day,obj_id


In [85]:
new_data = data.copy()
new_data.vote[new_data.vote==2] = 1
new_data.vote[new_data.vote==3] = 1
new_data.vote[new_data.vote==4] = 2
new_data.vote[new_data.vote==5] = 2
new_data.vote[new_data.vote==6] = 2
new_data.vote[new_data.vote==7] = 2
new_data.vote[new_data.vote==8] = 3
new_data.vote[new_data.vote==9] = 3
new_data.vote[new_data.vote==10] = 3

min_border_day = 15
min_border_obj = 116
threshold = 0.76819924

In [86]:
new_data.head()

,obj_id,hour,day,vote,ip,old_user
0,1719,1905,79,2,1,5
1,3309,1904,79,3,3,5
2,2038,1903,79,3,4,5
3,225,1907,79,3,5,0
4,2009,1905,79,2,6,5


In [82]:
data.head()

,obj_id,hour,day,vote,ip,old_user
0,1719,1905,79,5,1,5
1,3309,1904,79,9,3,5
2,2038,1903,79,8,4,5
3,225,1907,79,8,5,0
4,2009,1905,79,7,6,5


In [50]:
data_obj.head()

,obj_id,hour,day,vote,ip,old_user
995,16,1908,79,1,728,4
2295,16,1909,79,1,1413,5
5832,16,324,13,1,1859,5
17950,16,344,14,1,8487,5
22241,16,350,14,1,9844,3


In [61]:
counts_days.head()

,day,obj_id,vote,vote_x,rate_1
0,30,16,481,481,1.0
1,31,16,395,395,1.0
2,32,16,192,192,1.0
3,33,16,120,120,1.0
4,34,16,101,101,1.0


In [78]:
data_obj

,obj_id,hour,day,vote,ip,old_user
0,2,2,2,2,2,2
4,2,2,2,2,2,2
7,2,2,2,2,2,2
9,2,2,2,2,2,2
10,2,2,2,2,2,2
12,2,2,2,2,2,2
14,2,2,2,2,2,2
15,2,2,2,2,2,2
17,2,2,2,2,2,2
19,2,2,2,2,2,2


In [79]:

    
answers = []

obj_id = 2
data_obj = new_data[new_data.obj_id == obj_id]
counts_days = data_obj.pivot_table(index=['day', 'obj_id'], values='vote', aggfunc='count').reset_index()
if np.max(counts_days.vote) > min_border_obj:
    data_obj_counts = data_obj.merge(counts_days, on=['day', 'obj_id'], how='inner')
    data_obj_counts = data_obj_counts[data_obj_counts.vote_y > min_border_day]
    counts_days = counts_days[counts_days.vote > min_border_day]
    
    group_1 = data_obj_counts[data_obj_counts.vote_x == 1].pivot_table(index=['day', 'obj_id'], values='vote_x', aggfunc='count').reset_index()
    data_obj_counts = data_obj_counts.merge(group_1, on=['day', 'obj_id'], how='inner')
    if(len(group_1) > 0) :
        counts_days = counts_days.merge(group_1, on=['day', 'obj_id'], how='inner')
        counts_days['rate_1'] = counts_days.vote_x / counts_days.vote

        feature = np.max(counts_days.rate_1)

        if feature > threshold:
            answers.append([obj_id])


In [30]:
data_obj_counts.head()

,obj_id,hour,day,vote_x_x,ip,old_user,vote_y,vote_x_y,vote_x_x,vote_x_y
0,0,360,15,3,13031,2,46,2,12,32
1,0,361,15,2,13137,2,46,2,12,32
2,0,361,15,2,2195,2,46,2,12,32
3,0,365,15,3,601,1,46,2,12,32
4,0,365,15,2,13140,2,46,2,12,32


In [ ]:
	vote	vote_x	day	obj_id


Функция, которая напишет как происходит разделение в дереве.

In [140]:
def get_lineage(tree, feature_names):
    left = tree.tree_.children_left
    right = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features = [feature_names[i] for i in tree.tree_.feature]

    # id у child nodes
    idx = np.argwhere(left == -1)[:,0]     

    def recurse(left, right, child, lineage=None):          
        if lineage is None:
            lineage = [child]
        if child in left:
            parent = np.where(left == child)[0].item()
            split = 'l'
        else:
            parent = np.where(right == child)[0].item()
            split = 'r'

        lineage.append((parent, split, threshold[parent], features[parent]))

        if parent == 0:
            lineage.reverse()
            return lineage
        else:
            return recurse(left, right, parent, lineage)

    for child in idx:
        for node in recurse(left, right, child):
            print(node)

Обучаем решающее дерево

In [169]:
clf = DecisionTreeClassifier(max_depth=1)
clf.fit(all_features, all_answers)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [170]:
clf.feature_importances_

array([1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [171]:
get_lineage(clf, ['one', 'two', 'three', 'four', 'five', 'six'])

(0, 'l', 0.775188684463501, 'one')
1
(0, 'r', 0.775188684463501, 'one')
2


Получили, что самая важная фича - первая, то есть наибольшее отношение плохих отзывов ко всем среди всех дней

In [136]:
clf.tree_.threshold

array([ 0.76819924, -2.        , -2.        ])

In [172]:
min_border_day

15.0

In [173]:
min_border_obj

116.0

Пишем функцию детекции

In [186]:
def flashmob_detector(data):
    new_data = data.copy()
    new_data[new_data==2] = 1
    new_data[new_data==3] = 1
    new_data[new_data==4] = 2
    new_data[new_data==5] = 2
    new_data[new_data==6] = 2
    new_data[new_data==7] = 2
    new_data[new_data==8] = 3
    new_data[new_data==9] = 3
    new_data[new_data==10] = 3
    
    min_border_day = 15
    min_border_obj = 116
    threshold = 0.76819924
    
    answers = []
    
    for obj_id in tqdm(new_data.obj_id.unique()):
        data_obj = new_data[new_data.obj_id == obj_id]
        counts_days = data_obj.pivot_table(index=['day', 'obj_id'], values='vote', aggfunc='count').reset_index()
        if np.max(counts_days.vote) > min_border_obj:
            data_obj_counts = data_obj.merge(counts_days, on=['day', 'obj_id'], how='inner')
            data_obj_counts = data_obj_counts[data_obj_counts.vote_y > min_border_day]
            counts_days = counts_days[counts_days.vote > min_border_day]

            group_1 = data_obj_counts[data_obj_counts.vote_x == 1].pivot_table(index=['day', 'obj_id'], values='vote_x', aggfunc='count').reset_index()
            data_obj_counts = data_obj_counts.merge(group_1, on=['day', 'obj_id'], how='inner')

            counts_days = counts_days.merge(group_1, on=['day', 'obj_id'], how='inner')
            counts_days['rate_1'] = counts_days.vote_x / counts_days.vote

            feature = np.max(counts_days.rate_1)
            
            if feature > threshold:
                answers.append([obj_id])
   
    return answers

In [187]:
flashmob_detector(data)


 82%|████████▏ | 1895/2302 [00:37<00:05, 73.91it/s]


100%|██████████| 2302/2302 [00:44<00:00, 51.86it/s]

[[2800], [364], [598], [16], [453], [423], [792], [1710]]

In [188]:
true_answers

[792, 1710, 423, 598, 364, 453, 16]